In [1]:
data_dir = './data/mlboot_dataset/'
results_dir = './results/'
import pandas as pd
train = pd.read_table(data_dir + 'mlboot_data.tsv', header=None, names=['uid','cat','f1','f2','f3','days'], usecols=['uid','cat'])

In [2]:
import scipy.sparse as sp
import numpy as np

mat1 = sp.load_npz(data_dir+'mat1.npz')
mat2 = sp.load_npz(data_dir+'mat2.npz')
mat3 = sp.load_npz(data_dir+'mat3.npz')

In [3]:
train.cat.values.reshape(-1,1).shape

(19528597, 1)

In [4]:
mat = sp.hstack([train.cat.values.reshape(-1,1), mat1, mat2, mat3])
del train, mat1, mat2, mat3

In [5]:
from sklearn.decomposition import TruncatedSVD
print('pca on matrix 10')
pca = TruncatedSVD(n_components = 10, algorithm='arpack')
mat_pca10 = pca.fit_transform(mat.astype(np.float32))
print('pca on matrix 100')
pca = TruncatedSVD(n_components = 100, algorithm='arpack')
mat_pca100 = pca.fit_transform(mat.astype(np.float32))
#print('pca on matrix 300')
#pca = TruncatedSVD(algorithm='randomized', n_components = 100, n_iter=10)
#mat_pca300 = pca.fit_transform(mat.astype(np.float32))

pca on matrix 10
pca on matrix 100


In [6]:
import gc
del mat,pca
gc.collect()

0

In [ ]:
import tqdm
train = pd.read_table(data_dir + 'mlboot_data.tsv', header=None, names=['uid','cat','f1','f2','f3','days'], usecols=['uid','cat'])
lunique = train.uid.unique()

N = len(train)
M = len(lunique)
mask=sp.lil_matrix((M,N),dtype=np.int8)
row = -1
i = 0
last_uid = 'a'
for cuid in tqdm.tqdm(train.uid):
    if last_uid != cuid:
        last_uid = cuid
        row += 1
    mask[row,i] = 1
    i = i + 1

 37%|███▋      | 7157749/19528597 [00:31<00:53, 229543.54it/s]

In [ ]:
mat_pca10 = (mask*mat_pca10)
np.save(data_dir+'pca_dim10.npy', mat_pca10)
mat_pca100 = (mask*mat_pca100)
np.save(data_dir+'pca_dim100.npy', mat_pca100)